# Модульная контрольная работа №1
#Дедлайн: 20.03.2025 (четверг), 23:59
###Работа сдается в гугл-форму, которая будет закреплена в telegram конференции группы

###Работа при сдаче в гугл форму должна называться КР_Фамилия_Имя_Отчество.ipynb

###Перед сдачей просьба не стирать вывод ячеек для того, чтобы преподаватель заново не запускал ваш код

###Архив с данными прикладывать не нужно, только ipynb.

Задание.

1. Загрузить набор данных (https://www.kaggle.com/datasets/fabiochiusano/medium-articles) либо по архивом с конференции или гугл диск. Предстоит решение задачи мультилейбл классификации. В наборе данных содержится информация о статьях с Medium, целью является предсказание тематических тегов этих статей.
2. Произвести анализ данных, отобрать данные и метки, которые вы хотите/целесообразно использовать в процессе дообучения сети. После отбора в метках должно остаться не менее 25 уникальных тегов.
3. Подобрать модель для дообучения из открытых источников (huggingface подойдет, но можно и другие источники), а также генеративную модель, на которой будет решаться задача Zero-Shot мультилейбл классификация (к использованию подойдут как Encoder-Decoder архитектуры, так и Decoder-only).
4.  Выполнить Fine-Tunning выбранной вами модели, оценить ее качество, пользуясь метриками, подходящими для оценивания задачи мультилейбл классификации.
5.  Сравнить FT (Fine-Tuned) модель с 4-го пункта и выбранную вами Zero-Shot модель (модель, способную делать предсказания на новых данных без дообучения как такового), методы сравнения и метрики на ваше усмотрение.

#### P.S. Для Zero-Shot через Decoder-only лучше всего использовать Instruct модели
#### P.S.S. У вас всё получится, удачи :)

In [ ]:
import pandas as pd
from collections import Counter
import plotly.express as px
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!unzip /content/drive/MyDrive/archive.zip

Archive:  /content/drive/MyDrive/archive.zip
  inflating: medium_articles.csv     


# Анализ и подготовка данных

In [ ]:
df = pd.read_csv('/content/medium_articles.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/medium_articles.csv'

In [ ]:
df.head()

,title,text,url,authors,timestamp,tags
0,Mental Note Vol. 24,Photo by Josh Riemer on Unsplash\n\nMerry Chri...,https://medium.com/invisible-illness/mental-no...,['Ryan Fan'],2020-12-26 03:38:10.479000+00:00,"['Mental Health', 'Health', 'Psychology', 'Sci..."
1,Your Brain On Coronavirus,Your Brain On Coronavirus\n\nA guide to the cu...,https://medium.com/age-of-awareness/how-the-pa...,['Simon Spichak'],2020-09-23 22:10:17.126000+00:00,"['Mental Health', 'Coronavirus', 'Science', 'P..."
2,Mind Your Nose,Mind Your Nose\n\nHow smell training can chang...,https://medium.com/neodotlife/mind-your-nose-f...,[],2020-10-10 20:17:37.132000+00:00,"['Biotechnology', 'Neuroscience', 'Brain', 'We..."
3,The 4 Purposes of Dreams,Passionate about the synergy between science a...,https://medium.com/science-for-real/the-4-purp...,['Eshan Samaranayake'],2020-12-21 16:05:19.524000+00:00,"['Health', 'Neuroscience', 'Mental Health', 'P..."
4,Surviving a Rod Through the Head,"You’ve heard of him, haven’t you? Phineas Gage...",https://medium.com/live-your-life-on-purpose/s...,['Rishav Sinha'],2020-02-26 00:01:01.576000+00:00,"['Brain', 'Health', 'Development', 'Psychology..."


In [ ]:
df = df[['title', 'text', 'tags']]

In [ ]:
# Отбор 25 самых популярных тегов
all_tags = [tag for tags_list in df["tags"] for tag in eval(tags_list)]
d_tags_counter = Counter(all_tags)
tags, frequencies = list(zip(*d_tags_counter.most_common(n=25)))

fig = px.bar(x=tags, y=frequencies)
fig.update_xaxes(title="tags")
fig.update_yaxes(title="frequencies")
fig.show()

In [ ]:
relevant_tags = [
    "Blockchain", "Data Science", "Technology", "Programming", "Poetry",
    "Cryptocurrency", "Machine Learning", "Life", "Bitcoin", "Writing",
    "Politics", "Startup", "Life Lessons", "Self Improvement", "Covid 19",
    "Software Development", "Love", "Python", "Business", "Health",
    "Mental Health", "JavaScript", "Relationships", "Education", "Artificial Intelligence"
]

In [ ]:
import ast
def replace_with_relevant_tags(tags_list):
    if isinstance(tags_list, str):
        tags_list = ast.literal_eval(tags_list)

    matched_tags = [tag for tag in tags_list if tag in relevant_tags]
    return matched_tags if matched_tags else np.nan

df['tags'] = df['tags'].apply(replace_with_relevant_tags)

In [ ]:
df = df.dropna(subset=df.columns)

In [ ]:
df['num_tags'] = df['tags'].apply(lambda x: len(x) if isinstance(x, list) else 0)
print(df['num_tags'].value_counts())

num_tags
1    52746
2    21047
3     7077
4     1398
5      208
Name: count, dtype: int64


In [ ]:
# Для мультилейбл классификации лучше отфильтруем строки с одним тегом
df = df[df['num_tags'] > 1]

In [ ]:
# Создаем сбалансированную выборку из датасета
balanced_sample = []

for num_tag in [2, 3, 4, 5]:
    # Отбираем все строки с текущим количеством тегов
    subset = df[df['num_tags'] == num_tag]
    sample = subset.sample(n=100, random_state=42)
    balanced_sample.append(sample)

# Объединяем все выборки
final_df = pd.concat(balanced_sample, axis=0)

# Перемешиваем строки
df = final_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
df

,title,text,tags,num_tags
0,The pain of Two Factor Authentication(2FA),So are you trading now and your exchange added...,"[Technology, Startup, Bitcoin, Cryptocurrency]",4
1,Why Worry and Imagine the Worst?,I wrote this story three weeks ago in which I ...,"[Mental Health, Life Lessons, Love, Life]",4
2,The First Signs of Alcoholic Liver Damage Are ...,The First Signs of Alcoholic Liver Damage Are ...,"[Mental Health, Health]",2
3,"yes, this is an apocalypse — but it’s not what...","a few weeks ago, a few friends of mine were si...","[Health, Poetry, Covid 19, Mental Health]",4
4,5 Ways Government is Tackling the COVID-19 Dig...,Photo: Gabriel C. Pérez / KUT\n\nOver the cour...,"[Covid 19, Technology]",2
...,...,...,...,...
395,Is tech-veganism the trend of the future?,Avocado toast. Photo by Anna Pelzer.\n\nIs tec...,"[Startup, Technology]",2
396,Blockchain as a business technology of the fut...,Bitcoin and blockchain have been much talked a...,"[Technology, Blockchain, Bitcoin]",3
397,KuCoin Users Can Now Manage Their Portfolios W...,The people have spoken. They want Shrimpy with...,"[Blockchain, Technology, Bitcoin, Cryptocurrency]",4
398,A Full-Length Machine Learning Course in Pytho...,A Full-Length Machine Learning Course in Pytho...,"[Data Science, Machine Learning, Artificial In...",5


# Fine tuning

In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.8 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import torch
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('distilbert-base-uncased')
model = BertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=25, problem_type="multi_label_classification")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerN

In [ ]:
x = df['text']
yt = df['tags']

In [ ]:
from sklearn.model_selection import train_test_split
# First Split for Train and Test
x_train,x_test,y_train,y_test = train_test_split(x, yt, test_size=0.1, random_state=42,shuffle=True)
# Next split Train in to training and validation
x_tr,x_val,y_tr,y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42,shuffle=True)

In [ ]:
def tokenize_function(text):
    return tokenizer(text, padding='max_length', truncation=True, max_length=256)

# Применение токенизации к данным
x_train_tokenized = list(map(tokenize_function, x_train))
x_val_tokenized = list(map(tokenize_function, x_val))
x_test_tokenized = list(map(tokenize_function, x_test))

In [ ]:
mlb = MultiLabelBinarizer()
# Применяем one-hot encoding к меткам
y_train_encoded = mlb.fit_transform(y_train).astype('float32')
y_val_encoded = mlb.transform(y_val).astype('float32')
y_test_encoded = mlb.transform(y_test).astype('float32')

# Получаем список всех возможных тегов
all_tags = mlb.classes_

print("Encoded tags for a sample of y_train:")
print(y_train_encoded[:5])  # Показываем первые 5 примеров

print("All tags:", all_tags)  # Показываем все возможные теги

Encoded tags for a sample of y_train:
[[1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0.
  0.]
 [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0.
  0.]
 [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.
  0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1.
  0.]]
All tags: ['Artificial Intelligence' 'Bitcoin' 'Blockchain' 'Business' 'Covid 19'
 'Cryptocurrency' 'Data Science' 'Education' 'Health' 'JavaScript' 'Life'
 'Life Lessons' 'Love' 'Machine Learning' 'Mental Health' 'Poetry'
 'Politics' 'Programming' 'Python' 'Relationships' 'Self Improvement'
 'Software Development' 'Startup' 'Technology' 'Writing']


In [ ]:
# Создаем словарь для обучающего набора
train_data = {
    "input_ids": [item["input_ids"] for item in x_train_tokenized],
    "attention_mask": [item["attention_mask"] for item in x_train_tokenized],
    "labels": y_train_encoded.tolist()  # Преобразуем массив numpy в список
}

# Аналогично для валидационного набора
val_data = {
    "input_ids": [item["input_ids"] for item in x_val_tokenized],
    "attention_mask": [item["attention_mask"] for item in x_val_tokenized],
    "labels": y_val_encoded.tolist()
}

# Тестовый набор
test_data = {
    "input_ids": [item["input_ids"] for item in x_test_tokenized],
    "attention_mask": [item["attention_mask"] for item in x_test_tokenized],
    "labels": y_test_encoded.tolist()
}

# Преобразуем словари в объекты Dataset
train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(val_data)
test_dataset = Dataset.from_dict(test_data)

In [ ]:
# Создаем TrainingArguments — задаем параметры обучения
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    save_strategy="epoch"
)


# Создаем Trainer с подготовленными данными
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Запускаем процесс обучения
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning:

`evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead

<ipython-input-24-3635c194aa40>:17: FutureWarning:

`tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ayayanky (ayayanky-higher-school-of-economics) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,0.395434


TrainOutput(global_step=45, training_loss=0.42906231350368923, metrics={'train_runtime': 1249.7734, 'train_samples_per_second': 0.288, 'train_steps_per_second': 0.036, 'total_flos': 47369770168320.0, 'train_loss': 0.42906231350368923, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.39543426036834717,
 'eval_runtime': 67.7738,
 'eval_samples_per_second': 1.062,
 'eval_steps_per_second': 0.133,
 'epoch': 1.0}

In [ ]:
model.save_pretrained('./fine_tuned_bert')

In [ ]:
predictions = trainer.predict(test_dataset)

In [ ]:
pred_labels = np.argmax(predictions.predictions, axis=1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test_encoded.argmax(axis=1), pred_labels))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.00      0.00      0.00         7
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         3
           6       0.15      1.00      0.26         6
           9       0.00      0.00      0.00         2
          10       0.00      0.00      0.00         5
          16       0.00      0.00      0.00         1
          19       0.00      0.00      0.00         1
          22       0.00      0.00      0.00         2

    accuracy                           0.15        40
   macro avg       0.01      0.09      0.02        40
weighted avg       0.02      0.15      0.04        40



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Модель не успела нормально обучиться, но я уже не успеваю что-то переделать (нормальный fine tuning с нормальным кол-вом данных после 2 часов обучения истратил все ресурсы gpu и сбросился)

# zero shot

In [ ]:
!pip install transformers

     ---------------------------------------- 0.0/44.0 kB ? eta -:--:--
     --------- ------------------------------ 10.2/44.0 kB ? eta -:--:--
     -------------------------- ----------- 30.7/44.0 kB 262.6 kB/s eta 0:00:01
     -------------------------------------- 44.0/44.0 kB 309.9 kB/s eta 0:00:00
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.0 MB 1.9 MB/s eta 0:00:06
   ---------------------------------------- 0.1/10.0 MB 1.4 MB/s eta 0:00:07
   - -------------------------------------- 0.3/10.0 MB 2.5 MB/s eta 0:00:04
   -- ------------------------------------- 0.7/10.0 MB 4.2 MB/s eta 0:00:03
   ------ --------------------------------- 1.6/10.0 MB 7.2 MB/s eta 0:00:02
   ---------- ----------------------------- 2.7/10.0 MB 9.9 MB/s eta 0:00:01
   ---------------- ----------------------- 4.0/10.0 MB 12.8 MB/s eta 0:00:01
   -------

In [ ]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

C:\Users\yanki\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yanki\.cache\huggingface\hub\models--facebook--bart-large-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
# Тестируем только первый пример
first_text = x.iloc[0]  # Берем первый текст
true_label = yt.iloc[0]  # Истинная метка для первого текста

# Выполнение классификации для первого текстa
result = classifier(first_text, candidate_labels=relevant_tags, multi_label=True)

# Извлечение предсказанной метки
predicted_labels = []
predicted_labels.extend([label for label in result['labels'] if result['scores'][result['labels'].index(label)] >= 0.06])

In [ ]:
print("Истинные метки:", true_label)
print("Предсказанные метки:", predicted_labels)
print("Форма истинных меток:", len(true_label), "Форма предсказанных меток:", len(predicted_labels))

Истинные метки: ['Technology', 'Startup', 'Bitcoin', 'Cryptocurrency']
Предсказанные метки: ['Cryptocurrency', 'Health', 'Life', 'Startup']
Форма истинных меток: 4 Форма предсказанных меток: 4


Далее на cpu слишком долго считается, пришлось оставить так :(

In [ ]:
# Выполнение классификации в пакетном режиме
results = classifier(list(x), candidate_labels=relevant_tags)

# Извлечение предсказанных меток
predicted_labels = []
for result in results:
  predicted_labels.append([label for label in result['labels'] if result['scores'][result['labels'].index(label)] >= 0.06])

In [ ]:
predicted_labels = [set(pred) for pred in predicted_labels]  # Преобразуем в наборы для последующего сравнения
predicted_labels

In [ ]:
true_labels_set = [set(labels) for labels in yt]
true_labels_set

In [ ]:
from sklearn.metrics import f1_score
# Оценка метрик
# Используем макро F1-score для многометочной классификации
f1_macro = f1_score(true_labels_set, predicted_labels, average='macro')

print("F1 Score (Macro):", f1_macro)